In [4]:
import sqlite3

import pandas as pd
import numpy as np
from urllib.request import urlopen  
import json

getkey = ! cat ../../apikey.txt
apikey = getkey[0]

from fhelp import get_appended,process_datesIT

In [5]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [6]:
independents = urls[[name for name in urls.index if name[0] == 'G']]
dependents = urls[[name for name in urls.index if name[0] != 'G' and name[0] != 'U']]
unused = urls[[name for name in urls.index if name[0] == 'U']]

In [7]:
! rm ../../Finances.db
! rm schema.sql

In [8]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [9]:
conn = sqlite3.connect('../../Finances.db')
c = conn.cursor()
schemas = open("schema.sql", "a+")

In [10]:
%%time
for name, url in independents.iteritems():
    print(name,url)
    tab = get_appended(url,apikey,name)
    a = pd.io.sql.get_schema(tab,name)
    c.execute('drop table if exists '+ name)
    c.execute(a)
    tosql = a +';\n\n'
    schemas.write(tosql)
    tab.to_sql(name,con=conn,index=False,if_exists='append')

G1_CV_SYMBOLS https://financialmodelingprep.com/api/v3/stock/list?apikey=
G1_CV_ETFS https://financialmodelingprep.com/api/v3/etf/list?apikey=
G1_CV_TRADABLE https://financialmodelingprep.com/api/v3/available-traded/list?apikey=
G1_CV_DELISTED https://financialmodelingprep.com/api/v3/delisted-companies?limit=1000000000&apikey=
G1_CAL_EARNINGSCALENDAR_UPC https://financialmodelingprep.com/api/v3/earning_calendar?apikey=
G1_INST_CIKLIST https://financialmodelingprep.com/api/v3/cik_list?apikey=
G1_MI_MAJORINDEXES https://financialmodelingprep.com/api/v3/quotes/index?apikey=
G1_MI_SP500LIST https://financialmodelingprep.com/api/v3/sp500_constituent?apikey=
G1_MI_SP500HIST https://financialmodelingprep.com/api/v3/historical/sp500_constituent?apikey=
G1_MI_NASDAQLIST https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey=
G1_MI_NASDAQHISTLIST https://financialmodelingprep.com/api/v3/historical/nasdaq_constituent?apikey=
G1_MI_DOWJLIST https://financialmodelingprep.com/api/v3/dowj

In [11]:
#setup database
###FULL lists for full set processing
stocks = list(pd.io.sql.read_sql('select symbol from G1_CV_SYMBOLS;',con=conn).symbol)
etfs  = list(pd.io.sql.read_sql('select symbol from G1_ETF_SYMBOLS;',con=conn).symbol)
etfs2  = list(pd.io.sql.read_sql('select symbol from G1_CV_ETFS;',con=conn).symbol)
indexes = list(pd.io.sql.read_sql('select symbol from G1_MI_MAJORINDEXES;',con=conn).symbol)
ciks = ['000'+str(c) for c in list(pd.io.sql.read_sql('select cik from G1_INST_CIKLIST',con=conn).cik)]
funds = list(pd.io.sql.read_sql('select symbol from G1_MF_SYMBOLS;',con=conn).symbol)
commods = list(pd.io.sql.read_sql('select symbol from G1_COM_SYMBOLS;',con=conn).symbol)
##Light cleanup, nothing perfect –– could set this up using sql. 

#merge etf lists
etfs_c = list(set(etfs).union(set(etfs2)))
#leave stocks only 
stocks_c = list(set(stocks) - set(etfs_c + indexes + ciks + funds + commods))

In [12]:
#setup subset of tickers
dow_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_DOWJLIST;',con=conn).symbol)
nasdaq_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_NASDAQLIST;',con=conn).symbol)
sp500_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_SP500LIST;',con=conn).symbol)

In [13]:
#define subset to work with
stocksub = dow_tickers
indexsub = ['^IXIC','^GSPC','^DJI']

In [14]:
#get top US etfs proportionate to stock list
num_etfs = int(np.ceil(len(stocksub)/10))
profile = get_appended(dependents['P1b1000_CV_PROFILE'],apikey,'P1b1000_CV_PROFILE',tickers = etfs_c)
etfsub = list(profile[profile.country == 'US'].sort_values(by='volAvg',ascending=False).symbol[:num_etfs+1])
tickersub = stocksub + etfsub

In [15]:
#get mutual fund data
fundtable = get_appended(dependents['T1_INST_MUTFUNDHOLDERS'],apikey,'T1_INST_MUTFUNDHOLDERS',tickers=tickersub)
fundsub = list(fundtable.holder.value_counts().index)

In [16]:
mfs = pd.io.sql.read_sql('select * from G1_MF_SYMBOLS;',con=conn)
fundsub = list(np.random.choice(list(mfs[mfs.name.isin(fundsub)].symbol),int(np.ceil(len(stocksub)/3))))

In [17]:
#establish subset of commods
commodsub = commods[:3]

In [18]:
#get ciks of institutional holders for ciksub
instable = get_appended(dependents['T1_INST_HOLDERS'],apikey,'T1_INST_HOLDERS',tickers=tickersub)
instsub = list(instable.holder.value_counts().index)
instsub = list(np.random.choice(instsub,50)) #account for many missing inst-cik mappings

In [42]:
#retrieve ciks
ciklist = pd.io.sql.read_sql('select * from G1_INST_CIKLIST',con=conn)
ciksub = list(ciklist[ciklist.name.apply(lambda x: x.lower()).isin([i.lower() for i in instsub])].cik)
ciksub = ['000'+str(c) for c in (ciksub)]

In [43]:
#dictionary – nothing commodity, crypto, etc. specific
d = {'H':stocksub + etfsub + commods + fundsub,
     'P':stocksub + etfsub + fundsub,
     'T':stocksub + etfsub,
     'S':stocksub,
     'E':etfsub,
     'C':commodsub,
     'F':ciksub,
     'I':indexsub,
     'M':fundsub}

In [44]:
dependentsub = dependents.iloc[38:40]

In [45]:
#%%time
schemas = open("schema.sql", "a+")
for name, url in dependentsub.iteritems():
    print(name)
    if name[0] == 'U':
            continue 
    elif name[0]=='G': #if ticker info not needed
        tab = get_appended(url,apikey,name)
    else:
        tab = get_appended(url,apikey,name,tickers=d.get(name[0]))
    if len(tab) == 0: 
        print(name,' FAILED')
        #log error
        continue
    #load sql
    tablename = '_'.join(name.split('_')[1:])
    a = pd.io.sql.get_schema(tab,tablename)
    c.execute('drop table if exists '+ tablename)
    c.execute(a)
    tosql = a +';\n\n'
    schemas.write(tosql)
    tab.to_sql(tablename,con=conn,index=False,if_exists='append')

E1_INST_ETFCOUNTRYWEIGHTS


KeyboardInterrupt: 

HTTP Error 504: Gateway Time-out  for ticker 1454308, continuing.....
